In [1]:
import matplotlib
import numpy as np
import torch
import torch.nn as nn

import argparse
import os
import pickle
import random
import shutil
import time

from PIL import Image

In [2]:
adj = np.array([[1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 1., 0., 1., 1., 0., 1., 1., 0., 0., 0., 0.],
       [0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0.],
       [0., 0., 1., 1., 0., 1., 1., 0., 1., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 1., 0., 1., 1., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0.],
       [0., 0., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1.]])


neighbors_dict = {0: [1, 2],
                  1: [0, 2, 4, 5],
                  2: [0, 1, 3, 4, 5, 6],
                  3: [2, 5, 6],
                  4: [1, 2, 5, 7, 8],
                  5: [1, 2, 3, 4, 6, 7, 8, 9],
                  6: [2, 3, 5, 8, 9],
                  7: [4, 5, 8, 10],
                  8: [4, 5, 6, 7, 9, 10, 11],
                  9: [5, 6, 8, 10, 11],
                  10: [7, 8, 9, 11, 12],
                  11: [8, 9, 10, 12],
                  12: [10, 11]}

In [3]:
from torchvision import transforms, utils, models

to_tens = transforms.ToTensor()
# del final_im

for im in os.listdir("/home/hbaier/Desktop/graphPool/imagery"):
    cur_im = to_tens(Image.open(os.path.join("/home/hbaier/Desktop/graphPool/imagery", im)).convert("RGB")).unsqueeze(0)
    try:
        final_im = torch.cat((final_im, cur_im), dim = 0)
        # print("try")
    except Exception as e:
        final_im = cur_im
        # print(e)
    print(final_im.shape)

torch.Size([1, 3, 256, 256])
torch.Size([2, 3, 256, 256])
torch.Size([3, 3, 256, 256])
torch.Size([4, 3, 256, 256])
torch.Size([5, 3, 256, 256])
torch.Size([6, 3, 256, 256])
torch.Size([7, 3, 256, 256])
torch.Size([8, 3, 256, 256])
torch.Size([9, 3, 256, 256])
torch.Size([10, 3, 256, 256])
torch.Size([11, 3, 256, 256])
torch.Size([12, 3, 256, 256])
torch.Size([13, 3, 256, 256])


In [4]:
import matplotlib.pyplot as plt
from copy import deepcopy
import pandas as pd
import torch


class resnet18_mod(torch.nn.Module):

    def __init__(self, resnet):
        super().__init__()
        self.conv1 = resnet.conv1
        self.bn1 = resnet.bn1
        self.relu = resnet.relu
        self.maxpool = resnet.maxpool
        self.layer1 = resnet.layer1
        self.layer2 = resnet.layer2
        self.layer3 = resnet.layer3
        self.layer4 = resnet.layer4
        self.avgpool = resnet.avgpool
        self.fc = resnet.fc

    def forward(self, x):

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.maxpool(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.avgpool(out)
        out = out.flatten(start_dim=1)
        # out = torch.cat((out, ids, coords), dim = 1)
        # out = self.fc(out)
        return out

In [5]:
r18 = models.resnet18(pretrained = True)
model = resnet18_mod(r18)
input = model(final_im)#.detach().numpy()
input.shape

torch.Size([13, 512])

In [6]:
# GCN basic operation
class GraphConv(nn.Module):
    def __init__(self, input_dim, output_dim, bias=True):
        super(GraphConv, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.weight = nn.Parameter(torch.rand(input_dim, output_dim))#.cuda())
        if bias:
            self.bias = nn.Parameter(torch.rand(output_dim))#.cuda())
        else:
            self.bias = None

    def forward(self, x, adj):
        y = torch.matmul(adj, x)
        y = torch.matmul(y, self.weight)
        if self.bias is not None:
            y = y + self.bias
        return y

# Generate Assignment Matrix
class GenAssign(nn.Module):
    def __init__(self, input_dim, num_clusters, bias=True):
        super(GenAssign, self).__init__()
        self.input_dim = input_dim
        self.num_clusters = num_clusters
        self.weight = nn.Parameter(torch.rand(input_dim, num_clusters))#.cuda())
        if bias:
            self.bias = nn.Parameter(torch.rand(num_clusters))#.cuda())
        else:
            self.bias = None
        # self.linear = torch.nn.Linear(input_dim, num_clusters)
        self.sm = torch.nn.Softmax(dim = 1)

    def forward(self, x, adj):
        y = torch.matmul(adj, x)
        y = torch.matmul(y, self.weight)
        if self.bias is not None:
            y = y + self.bias
        # y = self.linear(y)
        y = self.sm(y)
        _, i = torch.max(y, 1)
        return i


# Pool based on clusters
class PoolClusters(nn.Module):
    def __init__(self):
        super(PoolClusters, self).__init__()

    def forward(self, x, sl):
        cluster_ids = torch.unique(sl)
        for cluster in cluster_ids:
            indices = torch.nonzero(sl == cluster, as_tuple = True)[0]
            cluster_mean = torch.mean(torch.index_select(x, 0, indices), dim = 0).unsqueeze(0)
            try:
                out = torch.cat((out, cluster_mean))
            except Exception as e:
                out = cluster_mean
        return out


def make_edge_list(neighbors_dict):
    edge_list = []
    for k,v in neighbors_dict.items():
        [edge_list.append([k, cur_v]) for cur_v in v]
    return edge_list

def make_adj_matrix(edge_list, dim):
    adj_matrix = np.zeros((dim, dim))
    for edge in edge_list:
        adj_matrix[edge[0]][edge[1]] = 1
    for i in range(dim):
        adj_matrix[i][i] = 1

    return adj_matrix


def fix_sl(sl):
    cluster_ids = torch.unique(sl)
    ref = dict(zip([i.item() for i in cluster_ids], [i for i in range(len(cluster_ids))]))
    for i in cluster_ids:
        sl[sl == i.item()] = ref[i.item()]
    return sl


# GCN basic operation
class GenAdj(nn.Module):
    def __init__(self):
        super(GenAdj, self).__init__()

    def forward(self, adj, sl, neighbors_dict):

        sl = fix_sl(sl)

        adj_map, new_neighbors_dict = {}, {}
        cluster_ids = torch.unique(sl)

        # Create a dictionary where each key is a node and the value is the cluster it belongs to
        for cluster in cluster_ids:
            indices = torch.nonzero(sl == cluster, as_tuple = True)[0]
            for i in indices:
                if i not in adj_map.keys():
                    adj_map[i.item()] = cluster.item()

        # Create a new neighbors dictionary for each of the clusters
        for cluster in cluster_ids:
            indices = torch.nonzero(sl == cluster, as_tuple = True)[0]

            for i in indices:
                temp = []
                [temp.append(adj_map[i]) for i in neighbors_dict[i.item()] if i not in temp]
                temp = list(set(temp)) # TO-DO: HERE CREATE SOME SORT OF SECONDARY DICTIONARY OR SOMETHING THAT COUNTS HOW MANY EDGES TO CREATE WEIGHTED ADJ MATRIX

                new_neighbors_dict[cluster.item()] = temp

        edge_list = make_edge_list(new_neighbors_dict)
        adj_matrix = make_adj_matrix(edge_list, len(cluster_ids))

        return torch.tensor(adj_matrix, dtype = torch.float32), new_neighbors_dict


class diffPool(nn.Module):
    def __init__(self, input_dim, output_dim, num_clusters, bias=True):
        super(diffPool, self).__init__()

        self.gc = GraphConv(input_dim = input_dim, output_dim = output_dim)
        self.bn = torch.nn.BatchNorm1d(output_dim)
        self.gen_assign = GenAssign(input_dim = output_dim, num_clusters = num_clusters)
        self.pool = PoolClusters()
        self.gen_adj = GenAdj()

    def forward(self, x, adj, neighbors_dict):

        # Generate node embeddings & normalize (no pooling yet)
        x = self.gc(x, torch.tensor(adj, dtype = torch.float32))
        x = self.bn(x)
        # Create the assignment matrix (i.e. assign each node to a cluster)
        sl = self.gen_assign(x, adj)
        # Create the pooled feature matrix
        x = self.pool(x, sl)
        # Generate the new adjacency matrix
        adj, neighbors_dict = self.gen_adj(adj, sl, neighbors_dict)
        # print(neighbors_dict)

        return x, adj, neighbors_dict

In [7]:
class diffPool(nn.Module):
    def __init__(self, input_dim, output_dim, num_clusters, bias=True):
        super(diffPool, self).__init__()

        self.gc = GraphConv(input_dim = input_dim, output_dim = output_dim)
        self.bn = torch.nn.BatchNorm1d(output_dim)
        self.gen_assign = GenAssign(input_dim = output_dim, num_clusters = num_clusters)
        self.pool = PoolClusters()
        self.gen_adj = GenAdj()

    def forward(self, x, adj, neighbors_dict):

        # Generate node embeddings & normalize (no pooling yet)
        x = self.gc(x, torch.tensor(adj, dtype = torch.float32))
        x = self.bn(x)
        # Create the assignment matrix (i.e. assign each node to a cluster)
        sl = self.gen_assign(x, adj)
        # Create the pooled feature matrix
        x = self.pool(x, sl)
        # Generate the new adjacency matrix
        adj, neighbors_dict = self.gen_adj(adj, sl, neighbors_dict)
        # print(neighbors_dict)

        return x, adj, neighbors_dict

In [8]:
class spatialPool(nn.Module):
    def __init__(self, input_dim, output_dim, num_clusters, resnet, bias=True):
        super(spatialPool, self).__init__()

        self.conv1 = resnet.conv1
        self.bn1 = resnet.bn1
        self.relu = resnet.relu
        self.maxpool = resnet.maxpool
        self.layer1 = resnet.layer1
        self.layer2 = resnet.layer2
        self.layer3 = resnet.layer3
        self.layer4 = resnet.layer4
        self.avgpool = resnet.avgpool

        self.dp1 = diffPool(input_dim = input_dim, output_dim = output_dim, num_clusters = 4)
        self.dp2 = diffPool(input_dim = output_dim, output_dim = 128, num_clusters = 2)
        self.linear = torch.nn.Linear(128, 1)

    def forward(self, x, adj, neighbors_dict):

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = x.flatten(start_dim=1)

        # print(x.shape)

        x, adj, neighbors_dict = self.dp1(x, adj, neighbors_dict)
        x, adj, neighbors_dict = self.dp2(x, adj, neighbors_dict)
        x = self.linear(x)

        return x

In [9]:
r18 = models.resnet18(pretrained = True)
model = spatialPool(input_dim = 512, output_dim = 256, num_clusters = 4, resnet = r18)
criterion = torch.nn.L1Loss(reduction = "mean")
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in range(0, 50):  

    pred = model(final_im, torch.tensor(adj, dtype = torch.float32), neighbors_dict)
    loss = criterion(pred, torch.tensor([10]))
    print(loss.item())

    optimizer.zero_grad()
    loss.backward()

    optimizer.step()

/home/hbaier/anaconda3/envs/caoe/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/home/hbaier/anaconda3/envs/caoe/lib/python3.7/site-packages/torch/nn/modules/loss.py:96: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([2, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
9.768965721130371
9.3634614944458
10.230894088745117
11.091619491577148
9.675439834594727
10.944665908813477
/home/hbaier/anaconda3/envs/caoe/lib/python3.7/site-packages/torch/nn/modules/loss.py:96: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results d

In [35]:
pred

tensor([[10.2095]], grad_fn=<AddmmBackward>)